In [215]:
# Packages
import requests, time, pandas as pd

In [236]:
# Global Variables
inc5000_url = "https://api.inc.com/rest/companyprofile/"
payload = ""
headers = {"cookie": "datadome=Z2il5N77MvwvffGojCx_2gvbXbTa-r2KgIOZ-GXA888eiFxK0MELmBs2akHERnMYnXaQ.l7zW8o-COaJA07abbgk4MTnj.9.0VGN-F~Te-7dmG6aUozOWJv6CSPZmQM; X-abtest=(null)"}
global_df = pd.DataFrame(columns = ['inc5000year','inc5000companyId','company_name',
                                    'founded','ceo_name','website','industry','business_model',
                                    'location','linkedin','twitter','facebook'])
error_counter = 0
error_log = []

In [227]:
# Read in 2022 CSV file (Prior Extract of INC5000 Data in CSV format)
inc_2022 = pd.read_csv(r'filepath')

In [228]:
# Local Storage Path - Export final DF as CSV here
local_storage_path = r'filepath'

In [229]:
def extract_csv(csv_file):
    company_ids = []
    for k in csv_file['inc5000companyId']:
        company_ids.append(str(k))
    return company_ids

In [230]:
def extract_transform_api(inc_id):

    global error_counter
    results = []
    url = inc5000_url + str(inc_id)
    response = requests.request("GET", url, data=payload, headers=headers)
    data = response.json()

    for i in data['profiles']:
        try:
            if i['inc5000companyId'] == inc_id:
                #print('inc5000 extract: ',i['inc5000companyId'])
                location = {'state': i['state_l'], 'city':i['ifc_city'],'zipcode':i['zipcode']}
                company_info = {
                'inc5000year' : 2022,
                'inc5000companyId': i['inc5000companyId'],
                'company_name' : i['ifc_company'],
                'founded' : i['founded'],
                'ceo_name': i['ifc_ceo_name'],
                'website': i['ifc_url'],
                'industry' : i['ifi_industry'],
                'business_model' : i['ifc_business_model'],
                'location' : location,
                'linkedin': i['ifc_linkedin_address'],
                'twitter': i['ifc_twitter_handle'],
                'facebook': i['ifc_facebook_address']
            }
                results.append(company_info)
        except KeyError:
            error_log.append({'id':inc_id})
            error_counter += 1
            print('Error Count: ', error_counter)
            continue
        finally:
            continue

    df = pd.DataFrame.from_dict(results)

    return df

In [232]:
def load_csv(df):
    print('Load CSV started...')
    df.to_csv(local_storage_path,index=False,header=True)
    print('CSV created...')

In [237]:
def avg_duration(l):
    total = 0
    duration_len = len(l)
    for j in l:
        total += int(j)
    avg_time = round((total/duration_len),2)

    return avg_time

In [242]:
def main():

    results = []
    duration_log = []

    company_ids = extract_csv(inc_2022)
    progress = 0
    length = len(company_ids)
    start_time = time.time()
    total_duration = 0
    for k in company_ids:
        new_str = k.strip()
        results.append(extract_transform_api(new_str))
        progress += 1
        if progress%50 == 0:
            end_time = time.time()
            duration = round((end_time - start_time),2)
            total_duration += duration
            total_duration_m = round(total_duration/60,2)
            start_time = end_time
            duration_log.append(int(duration))
            duration_average = avg_duration(duration_log)
            # Average Duration / Duration per 50 rows extracted
            print(f'Duration (s): {duration} -- Average Duration (s): {duration_average} -- Total Duration (m): {total_duration_m}\n'
                  f'Count: {progress} -- Completion(%): {(round((progress/length),2)*100)}\n')

    results.append(global_df)
    final = pd.concat(results, ignore_index=True)
    load_csv(final)

    print('Done')

In [243]:
main()

Duration (s): 4.91 -- Average Duration (s): 4.0 -- Total Duration (m): 0.08
Count: 50 -- Completion(%): 1.0
Duration (s): 3.01 -- Average Duration (s): 3.5 -- Total Duration (m): 0.13
Count: 100 -- Completion(%): 2.0
Duration (s): 3.98 -- Average Duration (s): 3.33 -- Total Duration (m): 0.2
Count: 150 -- Completion(%): 3.0
Duration (s): 2.92 -- Average Duration (s): 3.0 -- Total Duration (m): 0.25
Count: 200 -- Completion(%): 4.0
Duration (s): 2.91 -- Average Duration (s): 2.8 -- Total Duration (m): 0.3
Count: 250 -- Completion(%): 5.0
Duration (s): 46.98 -- Average Duration (s): 10.0 -- Total Duration (m): 1.08
Count: 300 -- Completion(%): 6.0


In [189]:
# Preview / Log Errors
error_df = pd.DataFrame(error_log)
error_df.to_csv(r'filepath', index=False)
print(error_counter)
print(error_log)

7
[{'id': '145183'}, {'id': '145179'}, {'id': '135699'}, {'id': '135695'}, {'id': '135691'}, {'id': '135687'}, {'id': '88050'}]
